In [1]:
import numpy as np
import pandas as pd
from dobbel import dobbellogger

In [3]:
dob = dobbellogger()
dob.log(10, 200, 4, 125)
dob.download()
data = dob.datadf

Logging data for 10s
Done! The data is located in self.datadf


In [4]:
def expq(vector, multiplier=1):
    x, y, z = vector[0] * multiplier, vector[1] * multiplier, vector[2]  * multiplier
    absq = (x ** 2 + y ** 2 + z ** 2) ** 1/2
    w = np.cos(absq)
    sinabs = np.sin(absq)
    xq = x * sinabs / absq
    yq = y * sinabs / absq
    zq = z * sinabs / absq
    return np.array([w, xq, yq, zq])

In [5]:
def quat_mul(p, q):
    p0 = p[0]
    q0 = q[0]
    pv = p[1:]
    qv = q[1:]
    out0 = p0 * q0 - np.dot(pv, qv)
    outv = p0 * qv + q0 * pv + np.cross(pv, qv)
    out = np.array([out0, *outv])
    return out

In [6]:
def quaternion_conjugate(q):
    w, x, y, z = q
    return np.array([w, -x, -y, -z])

In [7]:
# Actual Rotation Fuction - Input = Acceleration vector = 1x3 , rotation_quaternion = 1*4
def rotate_vector(vector, rotation_quaternion):
    # Convert the vector to a quaternion
    vector_quaternion = np.concatenate(([0], vector))

    # Calculate the rotated quaternion
    rotated_quaternion = quat_mul(rotation_quaternion, quat_mul(vector_quaternion, quaternion_conjugate(rotation_quaternion)))

    # Extract the rotated vector from the quaternion
    rotated_vector = rotated_quaternion[1:]

    return np.array(rotated_vector)

In [8]:
def quat_to_euler(q):
    w, x, y, z = q[0], q[1], q[2], q[3]
    psi = np.arctan((2*x*y-2*w*z)/((2*w**2)+(2*x**2)-1))
    theta = -np.arcsin(2*x*z+2*w*y)
    phi = np.arctan((2*y*z-2*w*x)/((2*w**2)+(2*z**2)-1))
    return np.array([psi, theta, phi])

In [9]:
def remove_nan(df):
    counter = 0
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter += 1
    for i in range(counter - 1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    counter = len(df) - 1
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter -= 1
    maxlen = len(df)
    for i in range(maxlen - 1, counter + 1, -1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df['timestamp'][i] = df['timestamp'][i] - df['timestamp'][0]
    for i in range(len(df)):
        for column in df.columns:
            if np.isnan(df[column][i]):
                deler = (df['timestamp'][i] - df['timestamp'][i - 1]) / (df['timestamp'][i + 1] - df['timestamp'][i - 1])
                df[column][i] = df[column][i - 1] + deler * (df[column][i + 1] - df[column][i - 1])
    return df

In [10]:
def left_quat_mul(q):
    q0, q1, q2, q3 = q
    lqm = np.array([
        [q0, -q1, -q2, -q3],
        [q1, 1, -q3, -q2],
        [q2, q3, 1, -q1],
        [q3, -q2, q1, 1]
    ])
    return lqm

def right_quat_mul(q):
    q0, q1, q2, q3 = q
    lqm = np.array([
        [q0, -q1, -q2, -q3],
        [q1, 1, q3, q2],
        [q2, -q3, 1, q1],
        [q3, q2, -q1, 1]
    ])
    return lqm

In [11]:
angle = np.deg2rad(-39)
rotate_y = np.array([[np.cos(angle),0,np.sin(angle)], [0,1,0],[-np.sin(angle),0,np.cos(angle)]])
trace = np.trace(rotate_y)
q_0 = np.sqrt(1+trace)/2
q_2 = 1/(4*q_0) * (np.sin(angle)+np.sin(angle))
q_rotate = np.array([q_0,0,q_2,0])
print(q_rotate)

[ 0.94264149  0.         -0.33380686  0.        ]


In [12]:
data = remove_nan(dob.datadf)

for i, row in data.iterrows():
    data.loc[i, 'x_acc'], data.loc[i, 'y_acc'], data.loc[i, 'z_acc'] = rotate_vector(np.array([row['x_acc'], row['y_acc'], row['z_acc']]), q_rotate)
    data.loc[i, 'x_gyro'], data.loc[i, 'y_gyro'], data.loc[i, 'z_gyro'] = rotate_vector(np.array([row['x_gyro'], row['y_gyro'], row['z_gyro']]), q_rotate)

y11 = np.array([data['x_acc'][0], data['y_acc'][0], data['z_acc'][0]])
gb = y11 / np.linalg.norm(y11)
gn = np.array([0, 0, 1])
A = - np.matmul(left_quat_mul(np.array([0, *gn])), right_quat_mul(np.array([0, *gb])))



# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(A)

# Eigenvalues are sorted in ascending order by np.linalg.eigh, 
# and the corresponding eigenvectors are columns of 'eigenvectors'
max_eigenvalue_index = np.argmax(eigenvalues)
max_eigenvector = eigenvectors[:, max_eigenvalue_index]


In [13]:
qtmin1tmin1 = max_eigenvector

dqde = np.array([
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1]
])

sigma_etha = np.array([
    [np.pi / 9, 0, 0],
    [0, np.pi / 9, 0],
    [0, 0, np.pi / 9]
])

Ptmin1tmin1 = 0.25 * np.matmul(np.matmul(np.matmul(np.matmul(right_quat_mul(qtmin1tmin1), dqde), sigma_etha), dqde.T), right_quat_mul(qtmin1tmin1))

In [14]:
gyro_bias = np.array([-0.145345332, 0.46132693, 0.01679937])
acc_bias = np.array([0.00682937, 0.0169035, -0.05057431])

qtmin1tmin1 = np.reshape(qtmin1tmin1, 4)

v = np.array([0, 0, 0])
p = np.array([0, 0, 0])
q_list = [qtmin1tmin1]
p_list = [p]
v_list = [v]

# std van gyroscope
Q = np.array([
    [-0.14534332, 0, 0],
    [0, 0.46132693, 0],
    [0, 0, 0.01679937]
])

# std van accelerometer
R = np.array([
    [-0.00014147, 0, 0],
    [0, 0.00180437, 0],
    [0, 0, 0.00452501],
])

for i in range(1, len(data['timestamp'])):
    dt = (data['timestamp'][i] - data['timestamp'][i - 1]) / 1000
    gyro_measurement = np.array([data['x_gyro'][i], data['y_gyro'][i], data['z_gyro'][i]])
    omega = gyro_measurement - gyro_bias
    expq_omega = expq(omega, 0.5 * dt * np.pi / 180)
    
    Ftmin1 = right_quat_mul(expq_omega)
    Gtmin1 = - 0.5 * dt * np.matmul(left_quat_mul(qtmin1tmin1), dqde)
    
    Pttmin1 = np.matmul(np.matmul(Ftmin1, Ptmin1tmin1), Ftmin1.T) + np.matmul(np.matmul(Gtmin1, Q), Gtmin1.T)
    
    qttmin1 = quat_mul(qtmin1tmin1, expq_omega)
    # G = - 0.5 * dt * np.matmul(left_quat_mul(qtmin1tmin1), dqde)
    
    q0, q1, q2, q3 = qttmin1
    # Zou kunnen dat hier de geconjugeerde genomen moet worden!!
    H = np.array([
        [2 * q2, 2 * q3, 2 * q0, 2 * q1],
        [-2 * q1, -2 * q0, 2 * q3, 2 * q2],
        [4 * q0, 0, 0, 4 * q3]
    ])
    S = np.matmul(np.matmul(H, Pttmin1), H.T) + R
    K = np.matmul(np.matmul(Pttmin1, H.T), np.linalg.inv(S))
    
    yat = np.array([data['x_acc'][i], data['y_acc'][i], data['z_acc'][i]]) - acc_bias
    yattmin1 = rotate_vector(gn, qttmin1) # Zou kunnen dat hier een min achter moet!!! en geconjugeerde
    epsilon = yat - yattmin1
    
    q_tt_measure = np.reshape(qttmin1 + np.matmul(K, epsilon), (4, 1))
    P_tt_measure = Pttmin1 - np.matmul(np.matmul(K, S), K.T)
    
    qtt = q_tt_measure / ((np.linalg.norm(q_tt_measure)))
    J = np.matmul(q_tt_measure, q_tt_measure.T) / (np.linalg.norm(q_tt_measure) ** 3)
    Ptt = P_tt_measure #np.matmul(np.matmul(J, P_tt_measure), J.T)
    
    Pttmin1 = Ptt
    qtmin1tmin1 = np.reshape(qtt, 4)
    
    q_list.append(np.reshape(qtt, 4))
    

In [15]:
for q, timestamp in zip(q_list, data['timestamp']):
    print(timestamp, quat_to_euler(q) * 180 / np.pi)

0.0 [-89.98891602  -2.00049654  -0.40036374]
25.0 [-89.9533519   -0.13593814   3.48642965]
29.0 [-8.99488038e+01 -4.70613041e-02  3.49572755e+00]
35.0 [-89.94285206   0.09417541   3.3812632 ]
40.0 [-8.99483963e+01 -8.79013549e-03  3.33161865e+00]
44.0 [-8.99441894e+01 -3.81945973e-02  3.58034442e+00]
45.0 [-8.99441149e+01 -2.66024411e-02  3.55584497e+00]
50.0 [-8.99403322e+01  3.02700356e-02  3.53970281e+00]
54.0 [-89.92921606   0.29647527   3.41913551]
60.0 [-8.99395837e+01  4.93135987e-02  3.43383663e+00]
64.0 [-89.93552536   0.13253427   3.41398049]
70.0 [-89.93430651   0.1585029    3.42888727]
75.0 [-89.94126933   0.12211431   3.23453461]
79.0 [-8.99470443e+01  4.34029356e-02  3.16174859e+00]
81.0 [-8.99460466e+01  2.74424552e-02  3.25302926e+00]
85.0 [-8.99439447e+01 -4.88186959e-03  3.41637120e+00]
89.0 [-89.93132895   0.16437846   3.56777476]
95.0 [-89.94397303   0.11383685   3.1297848 ]
100.0 [-8.99487185e+01 -5.39812048e-02  3.25069375e+00]
105.0 [-8.99397834e+01  8.80657774e-